# Testing Run Length Encoding


## Imports


In [8]:
from importlib.util import spec_from_loader, module_from_spec
from importlib.machinery import SourceFileLoader
from scipy.io import wavfile
from glob import glob
import numpy as np

# Import Encode
spec = spec_from_loader("encode", SourceFileLoader("encode", "../../encode"))
encode = module_from_spec(spec)
spec.loader.exec_module(encode)

# Import Decode
spec = spec_from_loader("decode", SourceFileLoader("decode", "../../decode"))
decode = module_from_spec(spec)
spec.loader.exec_module(decode)

## Function Definitions


In [9]:
def encode_via_rle(rle_l_raw: list):
    """This algorithm will search for contiguous values within the
       array. When the rle_location_count is greater than the value
       65530, the count is reduced by this value in order to
       prevent an overflow of an unsigned 16-bit integer. This allows
       for the data to be stored with 2 bytes when the format of the
       array is a known value in advance of decoding this format. The
       choice of integer 65530 is an arbitrary value less than that of
       the maximum value of an unsigned 16-bit integer (65536).

    Args:
        rle_l_raw (list): This is a list of integer values to be
                          encoded.

    Returns:

        index_array (list): This is the list of run length encoded
                            values.
        rle_locations (list): This is a list of locations of elements
                              that are repeated that are present in the
                              array of indices.
    """
    initial_index = 0
    second_index = 1
    count = 0
    index_array = []
    rle_locations = []
    rle_location_count = 0

    while second_index < len(rle_l_raw):
        if rle_l_raw[initial_index] == rle_l_raw[second_index]:
            index_array.append(rle_l_raw[initial_index])
            rle_locations.append(rle_location_count)

            # continue searching the breadth of the array; increasing
            # count
            while (
                second_index <= len(rle_l_raw)
                and rle_l_raw[initial_index] == rle_l_raw[second_index]
            ):
                count += 1
                second_index += 1
            index_array.append(count)
            if rle_location_count > 65530:
                rle_location_count -= 65530
            rle_location_count += 2
        else:
            index_array.append(rle_l_raw[initial_index])
            if rle_location_count > 65530:
                rle_location_count -= 65530
            rle_location_count += 1
        count = 0
        initial_index = second_index
        second_index += 1
    if rle_l_raw[-1] != index_array[-2]:
        index_array.append(rle_l_raw[-1])
    return index_array, rle_locations

In [10]:
def identify_index_split(rle_locations, verbose=False):
    """This will identify where the rle_location values reset to a value
       that is below 65530.
    Args:
        rle_locations (list): This is a list of locations of values that
                              are encoded in the index_array.

    Returns:
        split_indices (list): This is a list of indices where the values
                              are reset in the rle_locations_array.
    """
    split_indices = []
    for index, value in enumerate(rle_locations):
        if index == 0:
            prev_value = value
            continue
        if value > prev_value:
            prev_value = value
        else:
            if verbose:
                print(f"index: {index}")
                print(f"prev_value: {prev_value}")
                print(f"value: {value}")
            split_indices.append(index)
            prev_value = value
    return split_indices

In [11]:
# Expand RLE Algorithm
def decode_rle(rle_locations: list, index_array: list):
    """This will expand the index array where the values have been
       compressed by run-length-encoding.

    Args:
        rle_locations (list): This is a list of locations in the
                              index_array where the locations have been
                              run-length-encoded. The subsequent values
                              of these locations are frequencies of the
                              run-length-encoded values.
        index_array (list): This is a list of values which contain
                            either signular values or run-length-encoded
                            values followed by the frequency of the
                            run-length-encoded value.
    """
    reconstructed_array = []
    rle_index = 0
    continue_current_index_past_frequency_value = False

    for current_index in range(0, len(index_array)):
        if continue_current_index_past_frequency_value:
            continue_current_index_past_frequency_value = False
            continue
        if rle_index <= (len(rle_locations) - 1):
            if current_index > 131060:  # twice the cutoff value of 65530
                rle_location_index = rle_locations[rle_index] + 131060
            elif current_index > 65530 and current_index <= 131060:
                rle_location_index = rle_locations[rle_index] + 65530
            else:  # current_index is below 65530
                rle_location_index = rle_locations[rle_index]
        if current_index != rle_location_index:
            reconstructed_array.append(index_array[current_index])
        else:
            if rle_index <= (len(rle_locations) - 1):
                run_length_expanded_l = [
                    index_array[rle_location_index]
                    for x in range((index_array[rle_location_index + 1]) + 1)
                ]
                reconstructed_array.extend(run_length_expanded_l)
                rle_index += 1
                continue_current_index_past_frequency_value = True
    return reconstructed_array

In [12]:
def format_encoded_rle_to_bytes(index_array: list, rle_locations: list):
    """This is the encoding driver of the run_length_encoded array.

    Args:
        index_array (list): This is the compressed list of values which
                            include run-length-encoded values followed
                            by their frequency.
        rle_locations (list): This is a list of index locations that are
                              run-length-encoded in the index array.
                              The frequency of the run-length-encoded
                              value is defined as the
                              index_array[rle_locations[current_rle_index] + 1]
                              or the subsequent value from the
                              run-length-encoded value in the
                              index_array.

    Returns:
        format_encoded_rle_bytes (bytes): This is the string of bytes of
                                          the formatted
                                          run-length-encoded array.
    """
    format_encoded_rle = [len(rle_locations)]
    format_encoded_rle.extend(rle_locations)
    format_encoded_rle.extend(index_array)
    format_encoded_rle = np.array(format_encoded_rle, dtype=np.uint16)
    format_encoded_rle_bytes = format_encoded_rle.tobytes()
    return format_encoded_rle_bytes

In [30]:
def parse_formatted_rle_bytes(format_encoded_rle_bytes: bytes):
    """This is the encoding driver of the run_length_encoded array.

    Args:
        format_encoded_rle_bytes (bytes): This is the compressed list of
                                          values in the form of a byte
                                          array. The order of the values
                                          is as following: length of the
                                          run-length-encoded locations
                                          array, the run-length-encoded
                                          locations array, and the
                                          index_array.

    Returns:
        rle_locations (list): This is a list of unsigned 16-bit
                              integers.
        index_array (list): This is a list of unsigned 16-bit integers.
                            These values contain run-length-encoded
                            sequences where the run-length-encoded-value
                            is followed by the frequency of the value.
    """
    encoded_rle = np.frombuffer(format_encoded_rle_bytes, dtype=np.uint16)
    len_rle_locations = encoded_rle[0]

    # +1 to skip over the rle_locations length in the line below:
    rle_locations = encoded_rle[1 : len_rle_locations + 1]

    index_array = encoded_rle[(len_rle_locations + 1) :]
    return rle_locations, index_array

## Import Data & Huffman Encode Values


In [14]:
data_dir = "../../data/"
data_file_l = glob(data_dir + "*.wav")
current_file = data_file_l[0]

rate, data = wavfile.read(current_file)

data_bytes = data.tobytes()

compressed_file_path = "../../data/102b47d9-371e-412a-8995-0dc6115ab2bb.wav.brainwire"

encode.huffman_encoding(
    input_data=data_bytes,
    compressed_file_path="../../data/102b47d9-371e-412a-8995-0dc6115ab2bb.wav.brainwire",
)

with open(compressed_file_path, "rb") as fp:
    data_huffman_encoded_bytes = fp.read()
    fp.close()

# type(data_huffman_encoded_bytes)

In [15]:
data_huffman_encoded_bytes_sub_string = data_huffman_encoded_bytes

rle_l_raw = []
for data in data_huffman_encoded_bytes_sub_string:
    rle_l_raw.append(data)
rle_l_raw

[128,
 4,
 149,
 135,
 5,
 0,
 0,
 0,
 0,
 0,
 0,
 125,
 148,
 40,
 140,
 2,
 50,
 48,
 148,
 140,
 4,
 48,
 48,
 48,
 48,
 148,
 140,
 2,
 48,
 56,
 148,
 140,
 5,
 48,
 48,
 48,
 49,
 48,
 148,
 140,
 2,
 48,
 101,
 148,
 140,
 8,
 48,
 48,
 48,
 49,
 49,
 48,
 48,
 48,
 148,
 140,
 2,
 53,
 101,
 148,
 140,
 8,
 48,
 48,
 48,
 49,
 49,
 48,
 48,
 49,
 148,
 140,
 2,
 48,
 102,
 148,
 140,
 9,
 48,
 48,
 48,
 49,
 49,
 48,
 49,
 48,
 48,
 148,
 140,
 2,
 54,
 51,
 148,
 140,
 9,
 48,
 48,
 48,
 49,
 49,
 48,
 49,
 48,
 49,
 148,
 140,
 2,
 101,
 51,
 148,
 140,
 9,
 48,
 48,
 48,
 49,
 49,
 48,
 49,
 49,
 48,
 148,
 140,
 2,
 49,
 48,
 148,
 140,
 9,
 48,
 48,
 48,
 49,
 49,
 48,
 49,
 49,
 49,
 148,
 140,
 2,
 102,
 102,
 148,
 140,
 6,
 48,
 48,
 48,
 49,
 49,
 49,
 148,
 140,
 2,
 97,
 48,
 148,
 140,
 4,
 48,
 48,
 49,
 48,
 148,
 140,
 2,
 50,
 49,
 148,
 140,
 5,
 48,
 48,
 49,
 49,
 48,
 148,
 140,
 2,
 97,
 49,
 148,
 140,
 5,
 48,
 48,
 49,
 49,
 49,
 148,
 140,
 2,
 100,
 1

In [16]:
index_array, rle_locations = encode_via_rle(rle_l_raw)

In [17]:
reconstructed_array = decode_rle(rle_locations, index_array)

In [18]:
# Test for equality in reconstructed array
for index in range(0, len(rle_l_raw)):
    if rle_l_raw[index] != reconstructed_array[index]:
        print(f"Reconstructed Array is unequal at position: {index}")

In [19]:
formatted_encoded_rle_bytes = format_encoded_rle_to_bytes(index_array, rle_locations)

In [20]:
formatted_encoded_rle_bytes

b'\xcc\x03\x05\x00\x11\x00\x1b\x00\'\x00)\x00+\x005\x007\x009\x00D\x00F\x00J\x00T\x00V\x00d\x00f\x00i\x00t\x00v\x00y\x00~\x00\x83\x00\x85\x00\x8f\x00\x9b\x00\x9d\x00\xa8\x00\xaa\x00\xb6\x00\xd0\x00\xdb\x00\xdd\x00\xe8\x00\xec\x00\xf7\x00\x06\x01\t\x01\x14\x01 \x01"\x01-\x01:\x01E\x01P\x01\\\x01j\x01w\x01\x85\x01\x94\x01\x96\x01\xa4\x01\xa6\x01\xb3\x01\xc1\x01\xc7\x01\xcf\x01\xd2\x01\xdf\x01\xe2\x01\xe5\x01\xf2\x01\xf5\x01\xfd\x01\x05\x02\x08\x02\n\x02\x17\x02\'\x02)\x026\x02I\x02P\x02]\x02_\x02k\x02y\x02\x83\x02\x85\x02\x8f\x02\x91\x02\x9d\x02\x9f\x02\xa1\x02\xab\x02\xb8\x02\xbb\x02\xc5\x02\xc7\x02\xd1\x02\xdd\x02\xdf\x02\xe9\x02\xeb\x02\xf6\x02\xf8\x02\xfb\x02\x05\x03\x07\x03\x0c\x03\x16\x03\x18\x03\x1d\x03(\x03*\x031\x03;\x03=\x03D\x03P\x03R\x03Y\x03[\x03e\x03g\x03x\x03z\x03\x80\x03\x8a\x03\x8c\x03\x90\x03\x9a\x03\x9c\x03\x9e\x03\xa9\x03\xab\x03\xad\x03\xaf\x03\xb9\x03\xbb\x03\xbd\x03\xc1\x03\xcb\x03\xcd\x03\xcf\x03\xd3\x03\xde\x03\xe0\x03\xe2\x03\xe6\x03\xf1\x03\xf3\x03\xf5\x03\xf9\

In [21]:
encoded_rle = np.frombuffer(formatted_encoded_rle_bytes, dtype=np.uint16)

In [22]:
encoded_rle[0]

972

In [23]:
encoded_rle[1 : encoded_rle[0] + 1]

array([    5,    17,    27,    39,    41,    43,    53,    55,    57,
          68,    70,    74,    84,    86,   100,   102,   105,   116,
         118,   121,   126,   131,   133,   143,   155,   157,   168,
         170,   182,   208,   219,   221,   232,   236,   247,   262,
         265,   276,   288,   290,   301,   314,   325,   336,   348,
         362,   375,   389,   404,   406,   420,   422,   435,   449,
         455,   463,   466,   479,   482,   485,   498,   501,   509,
         517,   520,   522,   535,   551,   553,   566,   585,   592,
         605,   607,   619,   633,   643,   645,   655,   657,   669,
         671,   673,   683,   696,   699,   709,   711,   721,   733,
         735,   745,   747,   758,   760,   763,   773,   775,   780,
         790,   792,   797,   808,   810,   817,   827,   829,   836,
         848,   850,   857,   859,   869,   871,   888,   890,   896,
         906,   908,   912,   922,   924,   926,   937,   939,   941,
         943,   953,

In [24]:
encoded_rle[encoded_rle[0] + 1 :]

array([128,   4, 149, ..., 101,  46,  28], dtype=uint16)

In [25]:
rle_locations, index_array = parse_formatted_rle_bytes(formatted_encoded_rle_bytes)

In [26]:
reconstructed_array = decode_rle(rle_locations, index_array)

In [27]:
# Test for equality in reconstructed array
for index in range(0, len(rle_l_raw)):
    if rle_l_raw[index] != reconstructed_array[index]:
        print(f"Reconstructed Array is unequal at position: {index}")

In [33]:
len(formatted_encoded_rle_bytes)

262474

In [32]:
len(data_huffman_encoded_bytes)

130459

In [34]:
formatted_encoded_rle_bytes

b'\xcc\x03\x05\x00\x11\x00\x1b\x00\'\x00)\x00+\x005\x007\x009\x00D\x00F\x00J\x00T\x00V\x00d\x00f\x00i\x00t\x00v\x00y\x00~\x00\x83\x00\x85\x00\x8f\x00\x9b\x00\x9d\x00\xa8\x00\xaa\x00\xb6\x00\xd0\x00\xdb\x00\xdd\x00\xe8\x00\xec\x00\xf7\x00\x06\x01\t\x01\x14\x01 \x01"\x01-\x01:\x01E\x01P\x01\\\x01j\x01w\x01\x85\x01\x94\x01\x96\x01\xa4\x01\xa6\x01\xb3\x01\xc1\x01\xc7\x01\xcf\x01\xd2\x01\xdf\x01\xe2\x01\xe5\x01\xf2\x01\xf5\x01\xfd\x01\x05\x02\x08\x02\n\x02\x17\x02\'\x02)\x026\x02I\x02P\x02]\x02_\x02k\x02y\x02\x83\x02\x85\x02\x8f\x02\x91\x02\x9d\x02\x9f\x02\xa1\x02\xab\x02\xb8\x02\xbb\x02\xc5\x02\xc7\x02\xd1\x02\xdd\x02\xdf\x02\xe9\x02\xeb\x02\xf6\x02\xf8\x02\xfb\x02\x05\x03\x07\x03\x0c\x03\x16\x03\x18\x03\x1d\x03(\x03*\x031\x03;\x03=\x03D\x03P\x03R\x03Y\x03[\x03e\x03g\x03x\x03z\x03\x80\x03\x8a\x03\x8c\x03\x90\x03\x9a\x03\x9c\x03\x9e\x03\xa9\x03\xab\x03\xad\x03\xaf\x03\xb9\x03\xbb\x03\xbd\x03\xc1\x03\xcb\x03\xcd\x03\xcf\x03\xd3\x03\xde\x03\xe0\x03\xe2\x03\xe6\x03\xf1\x03\xf3\x03\xf5\x03\xf9\

In [35]:
data_huffman_encoded_bytes

b'\x80\x04\x95\x87\x05\x00\x00\x00\x00\x00\x00}\x94(\x8c\x0220\x94\x8c\x040000\x94\x8c\x0208\x94\x8c\x0500010\x94\x8c\x020e\x94\x8c\x0800011000\x94\x8c\x025e\x94\x8c\x0800011001\x94\x8c\x020f\x94\x8c\t000110100\x94\x8c\x0263\x94\x8c\t000110101\x94\x8c\x02e3\x94\x8c\t000110110\x94\x8c\x0210\x94\x8c\t000110111\x94\x8c\x02ff\x94\x8c\x06000111\x94\x8c\x02a0\x94\x8c\x040010\x94\x8c\x0221\x94\x8c\x0500110\x94\x8c\x02a1\x94\x8c\x0500111\x94\x8c\x02df\x94\x8c\x040100\x94\x8c\x02e1\x94\x8c\x0501010\x94\x8c\x02fd\x94\x8c\x0501011\x94\x8c\x025f\x94\x8c\x0501100\x94\x8c\x020c\x94\x8c\x070110100\x94\x8c\x0262\x94\x8c\x070110101\x94\x8c\x02fb\x94\x8c\x06011011\x94\x8c\x021f\x94\x8c\x0501110\x94\x8c\x02de\x94\x8c\x070111100\x94\x8c\x02f9\x94\x8c\x070111101\x94\x8c\x020a\x94\x8c\x06011111\x94\x8c\x02fe\x94\x8c\x0510000\x94\x8c\x0206\x94\x8c\x0510001\x94\x8c\x02e0\x94\x8c\x041001\x94\x8c\x0212\x94\x8c\x0b10100000000\x94\x8c\x025d\x94\x8c\x0b10100000001\x94\x8c\x021d\x94\x8c\x0b10100000010\x94\x8c\x0213

In [41]:
print(f"Ratio of the formatted_encoded_rle_bytes ", end="")
print(f"to data_huffman_encoded_bytes: ", end="")
print(f"{len(formatted_encoded_rle_bytes) / len(data_huffman_encoded_bytes)}")

Ratio of the formatted_encoded_rle_bytes to data_huffman_encoded_bytes: 2.011927118864931


#### Principal Components Example of the Run-Length-Encoded Problem


In [ ]:
# Original file size: 197 KB
# Huffman Encoded Compressed File Size: 130 KB
# RLE Encoded Version of the Huffman Encoded Compressed File is of Size: 272 KB

In [57]:
import sys

In [51]:
data_dir = "../../data/"
data_file_l = glob(data_dir + "*.wav")
current_file = data_file_l[0]

In [52]:
rate, data = wavfile.read(current_file)

data_bytes = data.tobytes()

compressed_file_path = "../../data/102b47d9-371e-412a-8995-0dc6115ab2bb.wav.brainwire"

encode.huffman_encoding(
    input_data=data_bytes,
    compressed_file_path="../../data/102b47d9-371e-412a-8995-0dc6115ab2bb.wav.brainwire",
)

with open(compressed_file_path, "rb") as fp:
    data_huffman_encoded_bytes = fp.read()
    fp.close()

In [58]:
sys.getsizeof(data)

197510

In [59]:
sys.getsizeof(data_bytes)

197431

In [50]:
test_array = np.array([1, 2, 3, 4], dtype=np.int16)
test_array

array([1, 2, 3, 4], dtype=int16)